# FM 5031

## HomeWork 4

Due Friday October 9



### MingxuanJiang

Collarborated with HaoChen, JingyiLi and HanlinDing

#### 1.  The $BEY$ swap curve in the Python cell below is observed on March 31, 2021. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt


termsKnown = np.array([1.0, 2.0, 5.0, 7.0, 10, 20, 30])
swapratesKnown = np.array([.05, .045, .035, .033, .04,.045, .047])

print(termsKnown, swapratesKnown)

[ 1.  2.  5.  7. 10. 20. 30.] [0.05  0.045 0.035 0.033 0.04  0.045 0.047]


a. Bootstrap this curve using your routine from Homework 1 or 2 (i.e., interpolating BEY's or assuming constant forwards).  Determine the fair coupon rate for a 10 in 5 forward starting swap.


In [2]:
# using the interpolating assumption
def simplebootstrap(mats, par_yields):
    
    
    #  This is not always needed, but sometimes helpful
    mats = np.array(mats)
    mats = mats.flatten()
    par_yields = np.array(par_yields)
    par_yields = par_yields.flatten()  
    #
    
    # transfer the type from float to int
    mats = mats.astype(int)

    firstmat = mats[0]
    lastmat = mats[-1]  #mats should be fed in in order.
    
    times = np.arange(0.5, lastmat+.5, 0.5)
    
    
    if mats[0] > 0.5:  #in case first par yield is not 6 months
        
        earlymats = np.arange(0.5,firstmat,.5)
        
        mats = np.append(earlymats,mats)
        
        earlypars = np.ones((1,2*firstmat -1)) * par_yields[0]
        earlypars = earlypars.flatten()
        par_yields = np.append(earlypars, par_yields)
        
    
    bey_interp = np.interp(times,mats,par_yields)
    
    # Make the coupon matrix
    
    C = np.kron(np.ones((len(bey_interp),1)),bey_interp).T /2
    
    
    C = np.tril(C) + np.eye(len(bey_interp))
    
    
    pars = np.ones((len(bey_interp),1))
    # solve matrix equation
    
    ZCB = np.matmul(np.linalg.inv(C),pars).flatten()
    
    
    spots = -np.log(ZCB) *  1/times
    
    return ZCB, spots
    

$$S(t;t_{start}, t_{end}) = 2\frac{Z(t,t_{start})- Z(t,t_{end})}{A_2(t,t_{start},t_{end})}$$

In [3]:
Z, rates = simplebootstrap(termsKnown, swapratesKnown)
c1 = 2*(Z[2*5-1]-Z[2*15-1])/sum(Z[10:30])
print('The fair coupon is',c1)

The fair coupon is 0.04750171207406646


b. The swap in a is entered into by Julia where Julia will pay floating and recieve fixed.  On March 31, 2022, the yield curve has changed and is given in the python code.  What is the value of Juila's swap?

In [4]:
termsKnown = np.array([1.0, 2.0, 5.0, 7.0, 10, 20, 30])
swapratesKnown2 = np.array([.02, .03, .05, .06, .075,.08, .09])

print(termsKnown, swapratesKnown2)

[ 1.  2.  5.  7. 10. 20. 30.] [0.02  0.03  0.05  0.06  0.075 0.08  0.09 ]


In [5]:
Z2, rates2 = simplebootstrap(termsKnown, swapratesKnown2)
# fixed leg
fixedleg = c1/2*sum(Z2[8:28])
print('Fixed leg',fixedleg)
# floating leg
floatingleg = Z2[7]-Z2[27]
print('Floating leg',floatingleg)
swapvalue = fixedleg - floatingleg
print('The value of swap is',swapvalue)

Fixed leg 0.24831907903418113
Floating leg 0.5270869185664864
The value of swap is -0.2787678395323052


#### 2. Use the same curves as in question 1.  On March 31 2021, Brad purches a 5 in 1 payor swaption at the money (meaning that the strike is the fair coupon rate of the corresponding foreward starting swap).

    1. On March 31,2022, what will the fair coupon rate for a five year swap be?

    2. What is the payoff on Brad's swaption?




A. On March 31,2022, what will the fair coupon rate for a five year swap be?

In [6]:
c2 = 2*(1-Z2[9])/sum(Z2[0:10])
print('The fair coupon is',c2)

The fair coupon is 0.049999999999999975


B. What is the payoff on Brad's swaption?

$$Swap(t;K,T_0,T_S)=MAX[S(t;T_O,T_S)-K,0]\frac{A_2(t,T_O,T_S)}{2}$$

In [7]:
c2b = 2*(Z[1]-Z[11])/sum(Z[2:12])
print('The fair coupon is',c2b)

The fair coupon is 0.030457493167406664


In [8]:
S = 2*(1-Z2[2*6-1-2])/sum(Z2[0:10])
print(S)
payoff2 = max(S-c2b,0)*sum(Z2[0:10])/2
print(payoff2)

0.049999999999999975
0.0877817354688057


#### 3. Suppose John wished to purchase a 20 in 5 payor swaption struck at $4.00\%$.  However, he can only purchase recievor swaptions, swaps, and forward starting swaps.  Use put-call parity to constuct a set of purchases equivalent to his desired purchase.  Specify the strikes, coupon rates , $T_O$, $T_S$, etc. Use the yield curve from problem one.

Comment: You can enter into a swap where the coupon is not the fair coupon. But in that case, a lump sum will need to be exchanged between the counter parties.

$$Swaption_{pay}(T_O;K,T_O,T_S)=\mathrm{Max}\left[S(T_O;T_O,T_S)-K, 0\right]\frac{A_2(T_O,T_O,T_S)}{2}$$
$$Swaption_{receive}(T_O;K,T_O,T_S)=\mathrm{Max}\left[K-S(T_O;T_O,T_S), 0\right]\frac{A_2(T_O,T_O,T_S)}{2}$$

In order to get a 20 in 5 payor swaption with strike 4%, we need to long a 20 in 5 receiver swaption with strike 4% and long a forward starting swap where pay 4% fixed rate and receive LIBOR for 20 years, starting in 5 years.

In [21]:
S = 2*(Z[5*2-1]-Z[25*2-1])/sum(Z[10:50])
c3 = 0.04
payoff3 = max(c3-S,0)*sum(Z[10:50])/2
print('The payoff of a 20 in 5 receiver swaption',payoff3)
#fixed leg
fixedleg3 = c3/2*sum(Z[10:50])
print('Fixed leg',fixedleg3)
# floating leg
floatingleg3 = Z[9]-Z[49]
print('Floating leg',floatingleg3)
swapvalue = fixedleg3 - floatingleg3
print('The value of a 20 in 5 receiver swaption is',swapvalue)

The payoff of a 20 in 5 receiver swaption 0.0
Fixed leg 0.42775044818893443
Floating leg 0.5413185359043637
The value of a 20 in 5 receiver swaption is -0.11356808771542931


#### 4. Write a 2 functions that calculate relevant mortgage information.

    1.  For a given rate, term, and payment, what is the loan amount?
    2.  For a given loan amount, payment, and term, what is the interest rate ?

$$L_m = L_{m-1}+interest_m-Payment$$
where
$$m \in \{0,1,2,\cdots,Terms\}$$

$$0=L_{Term}=L_0\times(1+r)^T-C\times\sum_{k=0}^{Term-1}(1+r)^k$$
$$L_0\times(1+r)^T-C\times\frac{(1+r)^T-1}{r}=0$$

A. For a given rate, term, and payment, what is the loan amount?

In [17]:
from scipy.optimize import fsolve
# r means monthly rate
# T means term
# C means payment
def GetLoanAmount(r,T,C):
    L0 = fsolve(lambda x: x*pow(1+r,T)-C*(pow(1+r,T)-1)/r,0)
    return L0

In [18]:
GetLoanAmount(0.005,360,1199.10)

array([199999.82481785])

B. For a given loan amount, payment, and term, what is the interest rate ?

In [19]:
# L0 means loan amount
# C means payment
# T means term
def GetInterestRate(L0,C,T):
    r = fsolve(lambda x: L0*pow(1+x,T) - C*(pow(1+x,T)-1)/x,0.01)
    return r
# this is a monthly rate

In [20]:
GetInterestRate(200000,1199.10,360)

array([0.00499999])